In [1]:
import sys
import numpy as np
import import_ipynb
from pathlib import Path
from scipy.sparse.linalg import eigsh

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from fem_processing import vectorial_matrices_assembly as assembly
    print("Modules imports were successful!")
except ModuleNotFoundError as en:
    print(f"Modules were not found: {en}")
except ImportError as en:
    print(f"Error in import: {en}")

Project root added to sys.path: C:\Users\adilt\OneDrive\05_GIT\PPGEE-2024.2-MEF
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!


# Project 4: Rectangular Guide

# `setTypeProblem()`

In [2]:
def setTypeProblem(FINITE_ELEMENT, h):
    # Create mesh using gmsh
    BOUNDARY = [{'tag': 101, 'type': 'Neumann', 'value': 0.0, 'name': 'Ez_0'}]
    MATERIAL = [{'tag': 201, 'name': 'free_space', 
                'relative_magnetic_permeability': 1, 'relative_electric_permittivity': 1}]
    
    # Create mesh Structure Data from gmsh
    mesh_data = ps.create_rectangular_guide(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, auto_save=False)

    # Get FEM solution
    Sgr, Mgr, _ = assembly.reduced_global_matrices(FINITE_ELEMENT, mesh_data)
    
    # Resolver o problema de autovalor generalizado
    eigenvalues, _ = eigsh(Sgr, k=35, M=Mgr, which='SM')

    # Definir um limiar para eliminar erros de truncamento
    positive_indices = np.where(eigenvalues > 1e-8)[0]

    # Filtrar autovalores positivos e ordenar
    sp_eigenvalues = eigenvalues[positive_indices]

    # Ordenar os autovalores positivos e ajustar os autovetores na mesma ordem
    sp_eigenvalues = sp_eigenvalues[np.argsort(sp_eigenvalues)]

    return sp_eigenvalues, mesh_data

# Convergence Analysis

In [3]:
element_types = [('Triangle', 1)]
mesh_sizes = [0.008 / (2**i) for i in range(0, 3)]

# Inicialização dos dicionários para armazenar erros acumulados para cada tipo de elemento
errors = {et: {} for et in element_types}
num_modes = {et: {} for et in element_types}

# Modos analíticos
tm_modes = ps.waveguide_modes(mn_max=5, mode_type='TM')

for h in mesh_sizes:  
    for FINITE_ELEMENT in element_types: 
        print(f"\nCalculating for Mesh size h={h} and element {FINITE_ELEMENT}.")
        # Resolve a solução numérica para o tipo de elemento e refinamento de malha
        eigenvalues, mesh_data = setTypeProblem(FINITE_ELEMENT, h)  

        # Preenche o dicionário dos autovalores
        num_modes[FINITE_ELEMENT][h] = eigenvalues

        # Acumula os erros nos dicionários correspondentes
        errors[FINITE_ELEMENT][h] = ps.calculate_error(tm_modes, eigenvalues)


5 menores autovalores analíticos dos modos TM:
TM_11: kc2 = 7710.628 rad2/m2
TM_21: kc2 = 12337.006 rad2/m2
TM_31: kc2 = 20047.634 rad2/m2
TM_12: kc2 = 26216.137 rad2/m2
TM_22: kc2 = 30842.514 rad2/m2

Calculating for Mesh size h=0.008 and element ('Triangle', 1).
Modo TM_11: Erro percentual = 0.0489
Modo TM_21: Erro percentual = 0.0671
Modo TM_31: Erro percentual = 0.1407
Modo TM_12: Erro percentual = 0.0849
Modo TM_22: Erro percentual = 0.0633

Calculating for Mesh size h=0.004 and element ('Triangle', 1).
Modo TM_11: Erro percentual = 0.0010
Modo TM_21: Erro percentual = 0.0091
Modo TM_31: Erro percentual = 0.0065
Modo TM_12: Erro percentual = 0.0187
Modo TM_22: Erro percentual = 0.0038

Calculating for Mesh size h=0.002 and element ('Triangle', 1).
Modo TM_11: Erro percentual = 0.0031
Modo TM_21: Erro percentual = 0.0046
Modo TM_31: Erro percentual = 0.0020
Modo TM_12: Erro percentual = 0.0044
Modo TM_22: Erro percentual = 0.0005


# Convergence Order, $\alpha$

In [4]:
lambdas = num_modes[('Triangle', 1)]
table = {mode: {} for mode in tm_modes.keys()}
for i, mode in enumerate(table.keys()):
    table[mode]['analytic'] = tm_modes[mode]    
    table[mode]['lh1'] = lambdas[mesh_sizes[0]][i]   
    table[mode]['lh2'] = lambdas[mesh_sizes[1]][i]   
    table[mode]['lh3'] = lambdas[mesh_sizes[2]][i]   
    table[mode]['alfa'] = np.log(
        np.abs(table[mode]['lh1'] - table[mode]['lh2'])/np.abs(table[mode]['lh2'] - table[mode]['lh3'])) / np.log(2)  
table

{'TM_11': {'analytic': 7710.628438351061,
  'lh1': 7714.397385542101,
  'lh2': 7710.552996219378,
  'lh3': 7710.870662809221,
  'alfa': 3.5971691761365383},
 'TM_21': {'analytic': 12337.0055013617,
  'lh1': 12345.288168185423,
  'lh2': 12335.880424910916,
  'lh3': 12337.578695090131,
  'alfa': 2.469782694672809},
 'TM_31': {'analytic': 20047.633939712763,
  'lh1': 20075.842493413205,
  'lh2': 20048.94338096872,
  'lh3': 20048.03409297919,
  'alfa': 4.886677463669662},
 'TM_12': {'analytic': 26216.136690393614,
  'lh1': 26193.872576802474,
  'lh2': 26221.02936608858,
  'lh3': 26217.29226921482,
  'alfa': 2.861323054459015},
 'TM_22': {'analytic': 30842.513753404244,
  'lh1': 30822.97897331943,
  'lh2': 30841.353995947855,
  'lh3': 30842.678602131164,
  'alfa': 3.794110623572509}}

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2025.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.